# Word2Vec - Unsupervised Model test

The purpose of thie ipynb is to test  the Word2Vec feature generation method on unsupervised learning model,  kmeans.

In [1]:
import nltk
from nltk.corpus import reuters
from nltk.corpus import stopwords
import pandas as pd
from gensim.models import Word2Vec
import gensim
import numpy as np
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
%matplotlib inline

## Dataset cleaning

•	We set a threshold of minimum 30 articles per category to be used as our dataset

•	Since, this is a text classification problem, we did further cleaning and preprocessing such as:
1.	HTML Character Replacement
2.	Removing Escape Sequences, such as newline or tab, and replacing it with a single space
3.	Removing punctuation and numbers as they do not provide any useful information
4.	removing commonly used words like 'the', 'and', 'in', etc, 
5.	and finally converts all words to lowercase

•	All this is done using Python built-in libraries and functions, and the Natural Language Toolkit 

•	Once the data is cleaned, it undergoes Word2Vec embeding as in 1.0.0. This is stored in new column called 'text_clean'.

In [2]:
stop_words = set(stopwords.words('english'))

# Example usage
documents = reuters.fileids()


In [3]:
#this creates a dataframe that splits it into 'category', 'subject' and 'content'
data = []

# Loop over each news article in the Reuters corpus
for article_id in documents:
    # Get the categories, title, and text of the article
    categories = reuters.categories(article_id)
    subject, body = reuters.raw(article_id).split('\n', maxsplit=1)
    if len(categories)>1:
        continue
    # Add a new row for each category
    for category in categories:
        # Store the data in a dictionary
        data.append({'category': category, 'subject': subject, 'content': body})

# Create a DataFrame from the data
reuters_df = pd.DataFrame(data)

# Show a few rows of the DataFrame
reuters_df

,category,subject,content
0,trade,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT,Mounting trade friction between the\n U.S. ...
1,grain,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS,A survey of 19 provinces and seven cities\n ...
2,ship,AUSTRALIAN FOREIGN SHIP BAN ENDS BUT NSW PORTS...,"Tug crews in New South Wales (NSW),\n Victo..."
3,gold,WESTERN MINING TO OPEN NEW GOLD MINE IN AUSTRALIA,Western Mining Corp Holdings Ltd\n &lt;WMNG...
4,acq,SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERGER,Sumitomo Bank Ltd &lt;SUMI.T> is certain to\...
...,...,...,...
9155,interest,FED SETS TWO BILLION DLR CUSTOMER REPURCHASE,The Federal Reserve entered the U.S.\n gove...
9156,earn,KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY,Qtly div 25 cts vs 25 cts prior\n Pay A...
9157,earn,TECHNITROL INC &lt;TNL> SETS QUARTERLY,Qtly div 12 cts vs 12 cts prior\n Pay A...
9158,earn,NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH QTR,Shr loss six cts vs loss 18 cts\n Net l...


In [4]:
reuters_df['text_clean'] = reuters_df['content'].apply(lambda x: gensim.utils.simple_preprocess(x))
reuters_df.head()

,category,subject,content,text_clean
0,trade,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT,Mounting trade friction between the\n U.S. ...,"[mounting, trade, friction, between, the, and,..."
1,grain,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS,A survey of 19 provinces and seven cities\n ...,"[survey, of, provinces, and, seven, cities, sh..."
2,ship,AUSTRALIAN FOREIGN SHIP BAN ENDS BUT NSW PORTS...,"Tug crews in New South Wales (NSW),\n Victo...","[tug, crews, in, new, south, wales, nsw, victo..."
3,gold,WESTERN MINING TO OPEN NEW GOLD MINE IN AUSTRALIA,Western Mining Corp Holdings Ltd\n &lt;WMNG...,"[western, mining, corp, holdings, ltd, lt, wmn..."
4,acq,SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERGER,Sumitomo Bank Ltd &lt;SUMI.T> is certain to\...,"[sumitomo, bank, ltd, lt, sumi, is, certain, t..."


In [5]:
#set minimum threshold for each category
threshold = 30

list_filteredCategory = reuters_df["category"].value_counts()[reuters_df["category"].value_counts()>threshold].index.tolist()
print(f"Number of categories with counts larger than {threshold} : {len(list_filteredCategory)}")
print(reuters_df["category"].value_counts()[reuters_df["category"].value_counts()>threshold])
# group the dataframe by the 'category' column and filter out the groups with counts less than the threshold
reuters_df = reuters_df.groupby('category').filter(lambda x: len(x) >= threshold)
reuters_df = reuters_df.reset_index(drop=True)
reuters_df

Number of categories with counts larger than 30 : 24
earn            3923
acq             2292
crude            374
trade            326
money-fx         309
interest         272
money-supply     151
ship             144
sugar            122
coffee           112
gold              90
gnp               74
cpi               71
cocoa             61
grain             51
alum              50
reserves          49
jobs              49
ipi               45
copper            44
rubber            40
iron-steel        38
nat-gas           36
bop               31
Name: category, dtype: int64


,category,subject,content,text_clean
0,trade,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT,Mounting trade friction between the\n U.S. ...,"[mounting, trade, friction, between, the, and,..."
1,grain,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS,A survey of 19 provinces and seven cities\n ...,"[survey, of, provinces, and, seven, cities, sh..."
2,ship,AUSTRALIAN FOREIGN SHIP BAN ENDS BUT NSW PORTS...,"Tug crews in New South Wales (NSW),\n Victo...","[tug, crews, in, new, south, wales, nsw, victo..."
3,gold,WESTERN MINING TO OPEN NEW GOLD MINE IN AUSTRALIA,Western Mining Corp Holdings Ltd\n &lt;WMNG...,"[western, mining, corp, holdings, ltd, lt, wmn..."
4,acq,SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERGER,Sumitomo Bank Ltd &lt;SUMI.T> is certain to\...,"[sumitomo, bank, ltd, lt, sumi, is, certain, t..."
...,...,...,...,...
8779,interest,FED SETS TWO BILLION DLR CUSTOMER REPURCHASE,The Federal Reserve entered the U.S.\n gove...,"[the, federal, reserve, entered, the, governme..."
8780,earn,KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY,Qtly div 25 cts vs 25 cts prior\n Pay A...,"[qtly, div, cts, vs, cts, prior, pay, april, r..."
8781,earn,TECHNITROL INC &lt;TNL> SETS QUARTERLY,Qtly div 12 cts vs 12 cts prior\n Pay A...,"[qtly, div, cts, vs, cts, prior, pay, april, r..."
8782,earn,NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH QTR,Shr loss six cts vs loss 18 cts\n Net l...,"[shr, loss, six, cts, vs, loss, cts, net, loss..."


In [6]:
len(reuters_df["text_clean"][0])

677

## Data preprocessing

The steps include:

**Label Encoding**: The LabelEncoder from sklearn is used to transform the categorical labels in the "category" column into numerical labels that can be used by machine learning algorithms. The transformed labels are stored in a new column 'label'.

**Train-Test Split**: The data is split into a training set and a test set, with 80% of the data used for training and 20% used for testing.

**Word2Vec Model Training**: A Word2Vec model is trained on the training set. This model generates a 100-dimensional vector for each word, considering a window of 5 words at a time. A word must appear at least twice in the corpus to be included in the model.

**Vectorization**: For each sentence in the training and test sets, the Word2Vec vectors of the words in the sentence are averaged to create a single vector representation of the sentence. This is done only for words that are present in the Word2Vec model's vocabulary.

**Handling Empty Vectors**: If a sentence doesn't contain any words present in the Word2Vec model's vocabulary, it results in an empty vector. To handle this, if a vector is empty, it's replaced with a zero vector of the same dimensionality as the Word2Vec vectors.

**Ensuring Matching Lengths**: It checks to ensure the length of the features (X_train_vect) matches the length of the target variable (y_train). If they don't match, it trims the longer one to match the length of the shorter one.


In [7]:
le = LabelEncoder()
reuters_df['label'] = le.fit_transform(reuters_df["category"])
#categories

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split (reuters_df['text_clean'], reuters_df['label'] , test_size=0.2)

In [8]:
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [9]:
# w2v_model.wv.index_to_key

In [9]:
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]).mean(axis=0)
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]).mean(axis=0)
                         for ls in X_test])

In [10]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [11]:
len(X_train_vect_avg)

7027

In [12]:
if len(y_train) < len(X_train_vect):
    X_train_vect = X_train_vect[:len(y_train)]
elif len(y_train) > len(X_train_vect):
    y_train = y_train[:len(X_train_vect)]


## kMeans testing

In [15]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix, normalized_mutual_info_score, precision_score, recall_score


In [17]:
kmeans = KMeans(n_clusters=24, random_state=42)
y_pred = kmeans.fit_predict(X_train_vect)
accuracy = accuracy_score(y_train, y_pred)

In [18]:
print(accuracy)

0.014373132204354632
